In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = keras.preprocessing.image_dataset_from_directory(
    directory = "../input/pleural-dataset/pleural_1600",
    label_mode = None,
    color_mode = "grayscale",
    image_size = (128,128),
    batch_size = 28,
    shuffle = True).map(lambda x: x/255.0)

Found 1600 files belonging to 1 classes.


2021-11-10 19:52:33.204497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 19:52:33.349407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 19:52:33.350492: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-10 19:52:33.357509: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(keras.Input(shape=(128,128,1)))
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dropout(0.5))
    
    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2D(1024, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())

    model.add(layers.Dropout(0.2))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

discriminator = make_discriminator_model()
discriminator.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 128)       3328      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 256)       0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 512)       3277312   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 512)       0

In [4]:
latent_dim = 100
generator = keras.Sequential(
    [
     layers.Input(shape=(latent_dim,)),
     layers.Dense(8*8*1024),
     layers.Reshape((8,8,1024)),
     layers.Conv2DTranspose(1024,kernel_size=5,strides=2,padding='same'),
     layers.LeakyReLU(0.2),
     layers.Conv2DTranspose(512,kernel_size=5,strides=2,padding='same'),
     layers.LeakyReLU(0.2),
     layers.Conv2DTranspose(256,kernel_size=5,strides=2,padding='same'),
     layers.LeakyReLU(0.2),
     layers.Conv2DTranspose(128,kernel_size=5,strides=2,padding='same'),
     layers.LeakyReLU(0.2),
     layers.Conv2DTranspose(1, kernel_size=5, padding='same',activation='tanh'),
    ]
)
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 65536)             6619136   
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 1024)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 1024)      26215424  
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16, 16, 1024)      0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 512)       13107712  
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 32, 512)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 64, 64, 256)      

In [5]:
opt_gen = keras.optimizers.Adam(1e-4)
opt_disc = keras.optimizers.Adam(1e-4)

loss_fn = keras.losses.BinaryCrossentropy()

In [6]:
os.mkdir("/kaggle/working/gen_imgs")
os.mkdir("/kaggle/working/training_checkpoints")
checkpoint_dir = '/kaggle/working/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=opt_gen,
                                 discriminator_optimizer=opt_disc,
                                 generator=generator,
                                 discriminator=discriminator)

In [7]:
gen_loss = []
disc_loss = []
for epoch in range(800):
    for idx, real in enumerate(tqdm(dataset)):
        batch_size = real.shape[0]

        random_latent_vectors = tf.random.normal(shape=(batch_size,latent_dim))
        fake = generator(random_latent_vectors)

        if idx % 100 == 0:
            img = keras.preprocessing.image.array_to_img(fake[0])
            img.save(f"/kaggle/working/gen_imgs/imgs_{epoch}_{idx}.png")

        with tf.GradientTape() as disc_tape:
            loss_disc_real = loss_fn(tf.ones((batch_size,1)), discriminator(real))
            loss_disc_fake = loss_fn(tf.zeros((batch_size,1)), discriminator(fake))
            loss_disc = (loss_disc_real+loss_disc_fake) / 2
            disc_loss.append(loss_disc)
        grads = disc_tape.gradient(loss_disc,discriminator.trainable_weights)
        opt_disc.apply_gradients(
            zip(grads, discriminator.trainable_weights))
        
        with tf.GradientTape() as gen_tape:
            fake = generator(random_latent_vectors)
            output = discriminator(fake)
            loss_gen = loss_fn(tf.ones(batch_size,1),output)
        grads = gen_tape.gradient(loss_gen,generator.trainable_weights)
        opt_gen.apply_gradients(
            zip(grads, generator.trainable_weights))
        gen_loss.append(loss_gen)
    #Saving after every 50 epoch
    print(f"Epoch : {epoch+1}")
    if (epoch + 1) % 200 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

  0%|          | 0/58 [00:00<?, ?it/s]2021-11-10 19:52:36.500587: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-10 19:52:39.317201: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
100%|██████████| 58/58 [01:21<00:00,  1.41s/it]


Epoch : 1


100%|██████████| 58/58 [00:38<00:00,  1.50it/s]


Epoch : 2


100%|██████████| 58/58 [00:38<00:00,  1.50it/s]


...

Epoch : 798


100%|██████████| 58/58 [00:38<00:00,  1.51it/s]


Epoch : 799


100%|██████████| 58/58 [00:40<00:00,  1.42it/s]


Epoch : 800
